In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Edit Distance

#### 1. Buatlah sebuah fungsi “edit_distance”. Fungsi tersebut memiliki dua buah parameter string yaitu string_1 dan string_2. Fungsi ini dapat menghitung Levenshtein Distance antara string_1 dengan string_2. Lebih jelasnya lagi, fungsitersebut akan mengembalikan nilai terkecil yang dibutuhkan untuk mentransformasi string_1 menjadi string_2.

In [2]:
def edit_distance(string_1, string_2):
    # menambahkan nilai awal selain indexnya dengan 0 
    matrix = np.zeros((len(string_1) + 1, len(string_2) + 1))

    for x in range(len(string_1) + 1):
        matrix[x][0] = x

    for y in range(len(string_2) + 1):
        matrix[0][y] = y
        
    # mencari bobot tiap huruf berdasarkan algoritma levenshtein distance
    for i in range (len(string_1) + 1):
        for j in range (len(string_2) + 1):
            if i == 0 or j == 0:
                matrix[i][j] = max(i, j)
            else:
                a = matrix[i - 1][j - 1]
                b = matrix[i][j - 1] + 1
                c = matrix[i - 1][j] + 1
       
                if string_1[i - 1] != string_2[j - 1]:
                    a += 1
       
                matrix[i][j] = min(a, b, c)
    return matrix[len(string_1)][len(string_2)]

#### 2. Menggunakan fungsi yang telah dibuat pada soal sebelumnya, carilah nilai edit_distance dari pasangan kata berikut ini:
<p>phasmophobia - puafnilhotik </p>
<p>genetik - ganteng</p>


In [3]:
print(edit_distance('phasmophobia','puafnilhotik'))
print(edit_distance('genetik','ganteng'))

7.0
5.0


#### 4. Menggunakan matriks M yang telah dibuat pada soal sebelumnya, carilah nilai edit_distance dari pasangan kata berikut ini:
<p>a. phasmophobia - puafnilhotik</p>
<p>b. genetik - ganteng</p>

In [5]:
total  = []
def keyboard_distance(max_overall,distance):
    total.append((max_overall-distance)/max_overall)

In [6]:
#a. phasmophobia - puafnilhotik
distance = [0,1,0,2,1,1,1,0,0,2,0,7]
for i in distance:
    keyboard_distance(9.75,i)
edit_distance = 0
for i in total:
    edit_distance += i
print("Edit distance phasmophobia - puafnilhotik adalah : " + str(edit_distance))

Edit distance phasmophobia - puafnilhotik adalah : 10.461538461538463


In [7]:
#b. genetik - ganteng
total  = []
distance = [0,2,0,2,2,2,3]
for i in distance:
    keyboard_distance(9.75,i)
edit_distance = 0
for i in total:
    edit_distance += i
print("Edit distance genetik - ganteng adalah : " + str(edit_distance))

Edit distance genetik - ganteng adalah : 5.871794871794871


#### 5. Bandingkan hasil yang kamu temui pada nomor A2 dan A4. Apa yang dapat kamu simpulkan terkait kedua metode edit distance ini? Selain itu, apakah kedua algoritma edit distance ini sudah sempurna? Jelaskan analisis singkat kamu minimal dalam 3 kalimat.

>Dari perbandingan hasil diatas terlihat bahwa keyboard distance memiliki hasil  edit distance yang lebih besar dibandingkan menggunakan Levenshtein Distance. Hal itu dikarenakan lavenshtein distance menghitung jumlah minimal perubahan antara satu string ke string lain sedangkan keyboard distance hanya melihat jumlah jarak antar karakter di keyboard yang tidak sama. Itu membuat menggunakan levenshtein distance lebih bagus dibandingkan keyboard distance untuk candidate generation. Hal itu karena kesalahan kata tidak hanya kesalahan penulisan(typo pada keyboard) namun bisa saja pengguna memang tidak tahu ejaan yang benar dari kata yang sudah diketik. Namun, menurut saya kedua algoritma edit distance ini tetap masih belum sempurna karena tidak akan menjamin akan menghasilkan keakuratan 100%.

#  N-Gram Language Model 

<p>Perhatikan empat kalimat berikut</p>
<p>K1 : orang itu menjadi kepala rumah sakit di desa</p>
<p>K2 : kepala orang itu sedang sakit di rumah sakit</p>
<p>K3 : kepala rumah sakit itu memakan kelapa bersama warga desa</p>
<p>K4 : orang sakit itu memakan kelapa bersama kepala rumah sakit</p>

#### 1. Jika diberikan threshold sebesar 10^-12 dengan menggunakan language model bigram dan trigram, Manakah di antara kalimat berikut ini yang memiliki kecenderungan pada spelling error? Tunjukkan peluang dari setiap kalimat berikut dan tentukan kalimat yang mengandung spelling error!
<p>a. warga desa sedang sakit kelapa di rumah sakit</p>
<p>b. orang itu sedang berada di rumah sakit</p>
<p>c. rumah sakit di desa itu tempat warga memakan kepala</p>
<p>d. orang sakit sedang memakan kelapa bersama warga desa<p>

In [65]:
all_sentence = "orang itu menjadi kepala rumah sakit di desa kepala orang itu sedang sakit di rumah sakit kepala rumah sakit itu memakan kelapa bersama warga desa  orang sakit itu memakan kelapa bersama kepala rumah sakit"
result_word = []
sentence = all_sentence.split()
for word in sentence:
    result_word.append(word)

word_df = pd.DataFrame(result_word, columns=["word"])
word_df_count = word_df["word"].value_counts().to_frame(name="count")

list_pword =[]
for i in word_df_count['count'] :
    p = i /len(result_word)
    list_pword.append(p)

In [66]:
#mendapatkan probabilitas setiap kata
word = ['sakit','rumah','itu','kepala','orang','bersama','kelapa','di','desa','memakan','menjadi','sedang','warga']        
df = pd.DataFrame(word, columns=["word"])
df['P(w)'] =  pd.Series(list_pword)
df

,word,P(w)
0,sakit,0.176471
1,rumah,0.117647
2,itu,0.117647
3,kepala,0.117647
4,orang,0.088235
5,bersama,0.058824
6,kelapa,0.058824
7,di,0.058824
8,desa,0.058824
9,memakan,0.058824


##### Bagian A 
##### a. warga desa sedang sakit kelapa di rumah sakit

<p>1. Bigram</p>
<p>P(a) = </p>
<p>p(warga) * P(desa | warga) * P(sedang | desa) * P(sakit | sedang) * P(kelapa | sakit) * p(di | kelapa) * P(rumah | di ) * P(sakit | rumah)

<p>2. Trigram</p>
<p>P(a) = </p>
<p>p(warga | *, *) * P(desa | *, warga) * P(sedang | warga, desa) * P (sakit | desa, sedang) * </p>
<p>P(kelapa | sedang, sakit) * P(di | sakit, kelapa) * P(rumah | kelapa, di) * P(sakit | di, rumah) </p>

#### P(warga | *, *) ama P(desa | *, warga) maksudnya peluang kata warga didepan kalimat sama peluang desa apabila sebelumnya kata warga yg didepan kalimat ####

In [71]:
#Bigram
total_a_bigram = 0.029412 * 1 * 1 * 1 * 0.5 * 1 * 1 * 1
print("a bigram : " + str(total_a_bigram))

#Trigram
total_a_trigram = 0.029412 * 0.029412  * 1 *  1 * 1 * 1 * 1 * 1
print("a trigram : " + str(total_a_trigram))

a bigram : 0.014706
a trigram : 0.000865065744


##### Bagian B
##### b. orang itu sedang berada di rumah sakit

<p>1. Bigram</p>
<p>P(b) = </p>
<p>p(orang) * P(itu | orang) * P(sedang | itu) * P(berada | sedang) * P(di | berada) * p(rumah | di) * P(sakit| rumah ) 

<p>2. Trigram</p>
<p>P(b) = </p>
<p>p(orang | *, *) * P(itu | *, orang) * P(sedang | orang, itu) * P (berada | itu, sedang) * </p>
<p>P(di | sedang, berada) * P(rumah | berada, di) * P(sakit | di, rumah) 

In [74]:
#Bigram
total_b_bigram = 0.088235 * (1/3) * (1) * (1/35) * (1/34) * 1 * (1/4)
print("b bigram : " + str(total_b_bigram))

#Trigram
total_b_trigram = 0.088235 * (1/3)  * 1 *  (1/35) * (1/34)  * (1/34) * 1 
print("b trigram : " + str(total_b_trigram))

b bigram : 6.17892156862745e-06
b trigram : 7.269319492502882e-07


##### Bagian C 
##### C. rumah sakit di desa itu tempat warga memakan kepala

<p>1. Bigram</p>
<p>P(a) = </p>
<p>p(rumah) * P(sakit | rumah) * P(di | sakit) * P(desa | di) * P(itu | desa) * p(tempat | itu) * P(warga | tempat ) * P(memakan | warga) * P(kepala | memakan)

<p>2. Trigram</p>
<p>P(a) = </p>
<p>p(rumah | *, *) * P(sakit | *, rumah) * P(di | rumah, sakit) * P (desa | sakit, di) * </p>
<p>P(itu | di, desa) * P(tempat | desa, itu) * P(warga | itu, tempat) * P(memakan | tempat, warga)*  P(kepala | warga, memakan)</p>

In [76]:
#Bigram
total_c_bigram = 0.117647 * (1/4) * (1) * (1) * (1/2) * (1/34) * (1/34)*1*1
print("c bigram : " + str(total_c_bigram))

#Trigram
total_c_trigram = 0.117647 * (1/4)  * 1 *  (1) * (1/34)  * (1/34) *(1/34)* (1/34)*(1/34)
print("c trigram : " + str(total_c_trigram))

c bigram : 1.2721345155709343e-05
c trigram : 6.473308139481652e-10


##### Bagian D
##### D. orang sakit sedang memakan kelapa bersama warga desa

<p>1. Bigram</p>
<p>P(a) = </p>
<p>p(orang) * P(sakit | orang) * P(sedang | sakit) * P(memakan | sedang) * P(kelapa | memakan) * p(bersama | kelapa) * P(warga | bersama ) * P(desa | warga)

<p>2. Trigram</p>
<p>P(a) = </p>
<p>p(orang | *, *) * P(sakit | *, orang) * P(sedang | orang, sakit) * P (memakan | sakit, sedang) * </p>
<p>P(kela[a | sedang, memakan) * P(bersama | memakan, kelapa) * P(warga | kelapa, bersama) * P(desa | bersama, warga)

In [78]:
#Bigram
total_d_bigram = 0.088235* (1/5) * (1) * (1/34) * (1/2) * (1/2) * (1/34)*1*1
print("d bigram : " + str(total_d_bigram))

#Trigram
total_d_trigram = 0.088235 * (1/5)  * (1/34) *  (1/34) * (1/34)  * (1/2) *(1)*1
print("d trigram : " + str(total_d_trigram))

d bigram : 3.816392733564013e-06
d trigram : 2.2449369020964784e-07


> Dari hasil diatas terbukti bahwa yang paling mendekati spelling  error adalah bagian c yaitu  rumah sakit di desa itu tempat warga memakan kepala

#### 2. Dalam kasus umum (tidak terbatas pada soal ini), menurut kamu, manakah yang lebih baik; language model unigram, bigram, atau trigram? 

1-gram (atau unigram) adalah urutan satu string. 2 gram (atau bigram) adalah rangkaian urutan dua string.  Dan 3-gram (atau trigram) adalah rangkaian urutan tiga string. Pada umumnya, semakin tinggi N, semakin baik atau akurat biasanya. Hal itu berarti trigram biasanya yang lebih baik . Tetapi hal tersebut mempunyai kelemahan karena dapat menyebabkan banyak overhead komputasi yang membutuhkan daya komputasi besar dalam hal RAM.

#### 3. Menurut kamu apakah N-gram language model yang standar sudah baik untuk diterapkan pada bahasa yang kaya secara morfologis, seperti Bahasa Indonesia? Apa saja masalah berkaitan dengan morfologis yang mungkin ditemukan pada kasus ini? Ceritakan strategi kamu dalam mengatasinya dikaitkan dengan pengetahuan yang telah didapatkan pada bagian Text Processing

Morfologi adalah ilmu yang mempelajari seluk-beluk kata serta fungsi perubahan-perubahan kata baik dalam fungsi gramatik (arti kata berdasarkan konteks penggunaan) maupun fungsi semantik (arti kata berdasarkan makna leksikal/kamus).
N-gram language model sendiri digunakan untuk memprediksi probabilitas dari urutan kata . N-gram language model mengambil potongan N-karakter yang diambilkan dari suatu string. Hal itu menyebabkan n-gram language model sendiri cukup menjadi standar yang baik untuk diterapkan pada bahasa secara morfologis dalam perubahan- perubahan kata. Namun, N-gram languange juga memiliki kelemahan seperti terbatasnya terhadap kata- kata yang tidak muncul sehingga adalah melakukan smoothing untuk
menghindari agar probabilitynya tidak 0. Selain itu, n-diagram juga mempunyai kelemahan seperti kurang tidak bekerja dengan biak dalam  longer-distance context.Oleh karena itu, language models yang lain seperti semantic indexing bekerja lebih baik.

# Pemetaan Dokumen dan Teks

#### 1. Buatlah pemetaan seluruh kata yang sudah dalam bentuk kata dasar lowercase dan bukan termasuk dalam stopwords dengan jumlah kemunculan kata tersebut di setiap dokumen.

In [8]:
document_a = "Sivitas Akademika Universitas Indonesia menjalankan aktivitas perkuliahan secara daring yang diikuti oleh mahasiswa dari seluruh jenjang. Hal ini diakibatkan karena adanya pandemi COVID-19 di Indonesia selama satu setengah tahun belakang."
document_b = "Masyarakat memiliki respon berbeda dalam menyikapi pandemi COVID-19. Salah satu mahasiswa bernama Dipsy mengatakan kesulitan untuk belajar dengan maksimal karena sulit untuk berdiskusi dengan orang yang lebih pandai."
document_c = "Salah satu kunci untuk menghadapi musibah adalah dengan bersabar dan yakin semua musibah akan berlalu pada waktunya. Badai pasti berlalu, jadi harus tetap semangat."

In [9]:
document_a = document_a.lower()
document_b = document_b.lower()
document_c = document_c.lower()
all_document = [document_a,document_b,document_c]

cachedStopWords = stopwords.words("indonesian")

#menghilangkan tanda baca
result_list=[] 
for i in all_document:
    r = re.findall('[^.\,"]',i)
    result = ''.join(r)
    result_list.append(result)

In [10]:
#tokenisasi kata dan menghilangkan stopwords
factory = StemmerFactory()
stemmer = factory.create_stemmer()

final_word =[]
for i in result_list:
    sent_word = nltk.word_tokenize(i)
    for word in sent_word:
        if word not in cachedStopWords:
             if word not in final_word:
                    output = stemmer.stem(word)
                    final_word.append(output)

In [11]:
result_dok_a = []
list_dok_a = result_list[0].split() 
for word in list_dok_a:
    if word in final_word:
        if word not in result_dok_a:
            result_dok_a.append(word)
print(result_dok_a)    

['sivitas', 'akademika', 'universitas', 'indonesia', 'aktivitas', 'daring', 'mahasiswa', 'jenjang', 'pandemi', 'covid-19']


In [12]:
result_dok_b = []
list_dok_b = result_list[1].split() 
for word in list_dok_b:
    if word in final_word:
        if word not in result_dok_b:
            result_dok_b.append(word)
print(result_dok_b)    

['masyarakat', 'respon', 'pandemi', 'covid-19', 'salah', 'mahasiswa', 'dipsy', 'maksimal', 'sulit', 'orang', 'pandai']


In [13]:
result_dok_c = []
list_dok_c = result_list[2].split() 
for word in list_dok_c:
    if word in final_word:
        if word not in result_dok_c:
            result_dok_c.append(word)
print(result_dok_c)    

['salah', 'kunci', 'musibah', 'badai', 'semangat']


In [14]:
word_df = pd.DataFrame(final_word, columns=["word"])
word_df['doc_A'] =  pd.Series([1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
word_df['doc_B'] =  pd.Series([0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0])
word_df['doc_C'] =  pd.Series([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1])
word_df

,word,doc_A,doc_B,doc_C
0,sivitas,1,0,0
1,akademika,1,0,0
2,universitas,1,0,0
3,indonesia,1,0,0
4,jalan,1,0,0
5,aktivitas,1,0,0
6,kuliah,1,0,0
7,daring,1,0,0
8,ikut,1,0,0
9,mahasiswa,1,1,0


#### 2. Jika menjalankan masing-masing query berikut, dokumen manakah yang akan ditemukan?
<p>a. akibat AND sulit
<p>b. waktu OR santai
<p>c. (masyarakat AND mahasiswa) OR sabar
<p>d. (libur OR respon) OR (waktu AND jenjang)
<p>e. (kunci OR musibah) AND NOT (aktivitas AND pandemi) OR maksimal

##### Jawab:
<p>a. Tidak akan menemukan dokumen apapun, karena dapat kita lihat dari hasil pemetaan seluruh kata tidak ada dokumen satupun yang memiliki kata akibat dan kata sulit</p>

<p>b. Tidak akan menemukan dokumen apapun, karena dapat kita lihat dari hasil pemetaan seluruh kata tidak ada dokumen satupun yang memiliki kata waktu atau kata santai. Selaain itu, waaktu merupakan stopword sehhingga tidak akan menghasilkan dokumen apapun</p>

<p>c. Dokumen B dan Dokumen C , karena dokumen B memiliki kata masyarakat dan mahasiswa sedangkan dokumen C memiliki kata sabar </p>

<p>d. Dokumen B , karena tidak ada dokumen yang mempunyai kata waktu dan jenjang</p>

<p>e. Dokumen C, karena tidak ada dokumen yang mempunyai (aktivitas AND pandemi) sehingga hanya memenuhi dokumen yang memiliki kata maksimal  </p>

#### 3. Apakah metode pemetaan dokumen dan teks dapat memberikan manfaat dalam proses perancangan sistem IR apabila dilihat dari perspektif keakuratan hasil yang akan dihasilkan? Jelaskan analisis singkat kamu minimal dalam 3 kalimat.

> Ya, menurut saya metode pemetaan dokumen dan teks dapat memberikan manfaat dalam proses perancangan sistem IR dan cukup akurat berdasarkan hasil yang dihasilkan. Walaupun keakuratan juga bergantung pada teknik memetakan antara kata-kata yang ada dengan kemunculanya pada satu atau beberapa dokumen namun cukup memberikan manfaat dalam perancangan sistem IR. Manfaat tersebut seperti mudah untuk dikembangkan dalam menghasilkan keakuratan yang cukup baik dan  merupakan struktur data paling populer yang digunakan dalam sistem IR.